<a href="https://colab.research.google.com/github/primagiant/playing-card-classification/blob/main/Card_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load and preprocess data
transform = transforms.Compose(
    [transforms.Resize((200, 200)), # resize images to 200x200
     transforms.ToTensor(), # convert images to tensors
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # normalize pixel values

batch_size = 4

trainset = torchvision.datasets.ImageFolder(root='cards-image-datasetclassification/14card types-14-(200 X 200)-94.61/train', transform=transform) # load training images from folder
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2) # create data loader for training set

testset = torchvision.datasets.ImageFolder(root='cards-image-datasetclassification/14card types-14-(200 X 200)-94.61/test', transform=transform) # load test images from folder
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2) # create data loader for test set

classes = ('ace', 'jack', 'king', 'queen', 'ten', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'joker') # define class names

In [ ]:
# Define model
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, 3), # convolution layer with 3 input channels and 32 output channels
    torch.nn.BatchNorm2d(32), # batch normalization layer
    torch.nn.ReLU(), # activation layer
    torch.nn.Conv2d(32, 64, 3), # convolution layer with 32 input channels and 64 output channels
    torch.nn.BatchNorm2d(64), # batch normalization layer
    torch.nn.ReLU(), # activation layer
    torch.nn.MaxPool2d(2), # max pooling layer with kernel size of 2
    torch.nn.Conv2d(64, 128, 3), # convolution layer with 64 input channels and 128 output channels
    torch.nn.BatchNorm2d(128), # batch normalization layer
    torch.nn.ReLU(), # activation layer
    torch.nn.Conv2d(128, 256, 3), # convolution layer with 128 input channels and 256 output channels
    torch.nn.BatchNorm2d(256), # batch normalization layer
    torch.nn.ReLU(), # activation layer
    torch.nn.Flatten(), # flatten layer to convert 4D tensor to 2D tensor
    torch.nn.Linear(256 * 47 * 47, 14) # linear layer with 256 * 47 * 47 input features and 14 output features
)

In [ ]:
# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # use Adam optimizer with learning rate of 0.001
criterion = torch.nn.CrossEntropyLoss() # use cross entropy loss for multi-class classification

In [ ]:
# Train model
num_epochs = 20 # number of epochs to train the model
for epoch in range(num_epochs):
    running_loss = 0.0 # initialize running loss to zero
    for i, data in enumerate(trainloader, 0): # loop over the training data loader
        inputs, labels = data # get the inputs and labels from the data loader
        optimizer.zero_grad() # zero the parameter gradients
        outputs = model(inputs) # forward pass: compute the outputs by passing inputs to the model
        loss = criterion(outputs, labels) # compute the loss using the outputs and labels
        loss.backward() # backward pass: compute the gradients of the loss with respect to model parameters
        optimizer.step() # update the model parameters using the gradients and optimizer

        running_loss += loss.item() # add the loss to the running loss
        if i % 2000 == 1999: # print statistics every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
# Save model
torch.save(model.state_dict(), 'model.pth') # save the model state dictionary to a file

In [ ]:
# Evaluate model
correct = 0 # initialize number of correct predictions to zero
total = 0 # initialize total number of predictions to zero
with torch.no_grad(): # disable gradient computation
    for data in testloader: # loop over the test data loader
        images, labels = data # get the images and labels from the data loader
        outputs = model(images) # forward pass: compute the outputs by passing images to the model
        _, predicted = torch.max(outputs.data, 1) # get the predicted class by finding the maximum value in the output tensor along dimension 1
        total += labels.size(0) # update the total number of predictions by adding the batch size
        correct += (predicted == labels).sum().item() # update the number of correct predictions by counting how many predictions match the labels

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total)) # print the accuracy by dividing correct by total and multiplying by 100